<a href="https://colab.research.google.com/github/srikanthrc/running-to-stand-still/blob/master/_notebooks/2020-03-21-covid19_overview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COVID-19 Tracking / Worldwide
> Tracking coronavirus total cases, deaths and new cases by country.  Separately, a view of US by state.

- author: Srikanth Chinmay (inspired by Pratap Vardhan @ http://covid19dashboards.com)
- categories: [covid-19]
- image: images/covid-overview.png
- permalink: /covid-overview/
- toc: false
- comments: false


In [1]:
#hide
import numpy as np
import pandas as pd
from datetime import datetime
from jinja2 import Template
from IPython.display import HTML

In [2]:
#hide
from pathlib import Path
if not Path('covid_overview.py').exists():
    ! wget https://raw.githubusercontent.com/srikanthrc/covid-19/master/covid_overview.py

--2020-06-26 11:52:21--  https://raw.githubusercontent.com/srikanthrc/covid-19/master/covid_overview.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9419 (9.2K) [text/plain]
Saving to: ‘covid_overview.py’

covid_overview.py   100%[===================>]   9.20K  --.-KB/s    in 0s      

2020-06-26 11:52:22 (84.6 MB/s) - ‘covid_overview.py’ saved [9419/9419]



In [3]:
#hide
import covid_overview as covid

In [4]:
#hide
COL_REGION = 'Country/Region'   

kpis_info = [
    {'title': 'Asia', 'prefix': 'Asia'},
    {'title': 'Europe', 'prefix': 'EU'},
    {'title': 'North America', 'prefix': 'NA'}]

data = covid.gen_country_data(kpis_info=kpis_info)

In [5]:
#hide
#display(data['table'].head(10))
data['summary']

{'Asia Cases': 2062328,
 'Asia Cases (+)': 43744,
 'Asia Deaths': 51675,
 'Asia Deaths (+)': 938,
 'Cases': 9609829,
 'Cases (+)': 178479,
 'Deaths': 489312,
 'Deaths (+)': 6554,
 'EU Cases': 2363490,
 'EU Cases (+)': 15113,
 'EU Deaths': 189484,
 'EU Deaths (+)': 422,
 'NA Cases': 2824801,
 'NA Cases (+)': 49780,
 'NA Deaths': 160466,
 'NA Deaths (+)': 3239,
 'since': Timestamp('2020-06-24 00:00:00'),
 'updated': Timestamp('2020-06-25 00:00:00')}

In [6]:
#hide_input
template = Template(covid.get_template(covid.paths['overview']))
dt_cols, LAST_DATE_I = data['dt_cols'], data['dt_last']

html = template.render(
    D=data['summary'], table=data['table'].head(30),
    newcases=data['newcases'].loc[:, dt_cols[LAST_DATE_I-50]:dt_cols[LAST_DATE_I]],
    KPIS_INFO=kpis_info,
    np=np, pd=pd, enumerate=enumerate
   )
HTML(f'<div>{html}</div>')

#####Visualizations by [Pratap Vardhan](https://twitter.com/PratapVardhan)[^1]

[^1]: Sources: ["COVID-19 Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub](https://github.com/CSSEGISandData/COVID-19). [covidtracking.com](https://covidtracking.com/)